In [1]:
import pandas as pd
import BlackScholes as bs
import variance_curve as vc
import risk_free_rates as rf
import implied_q as iq
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt

from scipy import interpolate
from scipy.optimize import least_squares as ls
from scipy.integrate import quad

## VIX data and functions

In [2]:
def horner_vector(poly, n, x):
    #Initialize result
    result = poly[0].reshape(-1,1)
    for i in range(1,n):
        result = result*x + poly[i].reshape(-1,1)
    return result

In [3]:
def gauss_dens(mu,sigma,x):
    return 1/np.sqrt(2*np.pi*sigma**2)*np.exp(-(x-mu)**2/(2*sigma**2))

In [4]:
def vix_futures(H, eps, T, a_k_part, k, r, q, n_steps):

    a2,a4 = (0,0)
    a0,a1,a3,a5 = a_k_part
    a_k = np.array([a0, a1, a2, a3, a4, a5])
    
    kappa_tild = (0.5-H)/eps
    eta_tild = eps**(H-0.5)

    delt = 30/365
    T_delta = T + delt

    std_X = eta_tild*np.sqrt(1/(2*kappa_tild)*(1-np.exp(-2*kappa_tild*T)))
    dt = delt/(n_steps)
    tt = np.linspace(T, T_delta, n_steps+1)
    
    FV_curve_all_vix = vc.variance_curve(tt)
    
    exp_det = np.exp(-kappa_tild*(tt-T))
    cauchy_product = np.convolve(a_k,a_k)
    
    std_Gs_T = eta_tild*np.sqrt(1/(2*kappa_tild)*(1-np.exp(-2*kappa_tild*(tt-T))))
    std_X_t = eta_tild*np.sqrt(1/(2*kappa_tild)*(1-np.exp(-2*kappa_tild*tt)))
    std_X_T = std_X
    
    n = len(a_k)
    
    normal_var = np.sum(cauchy_product[np.arange(0,2*n,2)].reshape(-1,1)*std_X_t**(np.arange(0,2*n,2).reshape(-1,1))*\
    scipy.special.factorial2(np.arange(0,2*n,2).reshape(-1,1)-1),axis=0) #g(u)
    
    beta = []
    for i in range(0,2*n-1):
        k_array = np.arange(i,2*n-1)
        beta_temp = ((std_Gs_T**((k_array-i).reshape(-1,1))*((k_array-i-1)%2).reshape(-1,1)*\
            scipy.special.factorial2(k_array-i-1).reshape(-1,1)*\
            (scipy.special.comb(k_array,i)).reshape(-1,1))*\
            exp_det**(i))*cauchy_product[k_array].reshape(-1,1)
        beta.append(np.sum(beta_temp,axis=0))

    beta = np.array(beta)*FV_curve_all_vix/normal_var
    beta = (np.sum((beta[:,:-1]+beta[:,1:])/2,axis=1))*dt
    
    sigma = np.sqrt(eps**(2*H)/(1-2*H)*(1-np.exp((2*H-1)*T/eps)))
    
    f = lambda x: np.sqrt(horner_vector(beta[::-1], len(beta), x)/delt)*100 * gauss_dens(0, sigma, x)

    Ft, err = quad(f, -np.inf, np.inf)
    
    return Ft * np.exp((r-q)*T)

In [5]:
def vix_iv(H, eps, T, a_k_part, K, r, q, n_steps):

    a2,a4 = (0,0)
    a0,a1,a3,a5 = a_k_part
    a_k = np.array([a0, a1, a2, a3, a4, a5])
    
    kappa_tild = (0.5-H)/eps
    eta_tild = eps**(H-0.5)

    delt = 30/365
    T_delta = T + delt

    std_X = eta_tild*np.sqrt(1/(2*kappa_tild)*(1-np.exp(-2*kappa_tild*T)))
    dt = delt/(n_steps)
    tt = np.linspace(T, T_delta, n_steps+1)
    
    FV_curve_all_vix = vc.variance_curve(tt)
    
    exp_det = np.exp(-kappa_tild*(tt-T))
    cauchy_product = np.convolve(a_k,a_k)
    
    std_Gs_T = eta_tild*np.sqrt(1/(2*kappa_tild)*(1-np.exp(-2*kappa_tild*(tt-T))))
    std_X_t = eta_tild*np.sqrt(1/(2*kappa_tild)*(1-np.exp(-2*kappa_tild*tt)))
    std_X_T = std_X
    
    n = len(a_k)
    
    normal_var = np.sum(cauchy_product[np.arange(0,2*n,2)].reshape(-1,1)*std_X_t**(np.arange(0,2*n,2).reshape(-1,1))*\
    scipy.special.factorial2(np.arange(0,2*n,2).reshape(-1,1)-1),axis=0) #g(u)
    
    beta = []
    for i in range(0,2*n-1):
        k_array = np.arange(i,2*n-1)
        beta_temp = ((std_Gs_T**((k_array-i).reshape(-1,1))*((k_array-i-1)%2).reshape(-1,1)*\
            scipy.special.factorial2(k_array-i-1).reshape(-1,1)*\
            (scipy.special.comb(k_array,i)).reshape(-1,1))*\
            exp_det**(i))*cauchy_product[k_array].reshape(-1,1)
        beta.append(np.sum(beta_temp,axis=0))

    beta = np.array(beta)*FV_curve_all_vix/normal_var
    beta = (np.sum((beta[:,:-1]+beta[:,1:])/2,axis=1))*dt
    
    sigma = np.sqrt(eps**(2*H)/(1-2*H)*(1-np.exp((2*H-1)*T/eps)))
    
    N = len(K); P = np.zeros(N);
    
    for i in range(N):
    
        f = lambda x: np.maximum(np.sqrt(horner_vector(beta[::-1], len(beta), x)/delt)*100 - K[i], 0) * gauss_dens(0, sigma, x)
        P[i], err = quad(f, -np.inf, np.inf)
    
    return P * np.exp((r-q)*T)

In [6]:
data = pd.read_csv("vix_opt.csv")
vix_T = np.array(data["Exp Date"])[2:13]
vix_f = np.array(data["Futures"])[2:13]
vix_IV = np.array(data.iloc[:,2:])[2:13,:]/100
vix_k = np.array([80, 90, 95, 97.5, 100, 102.5, 105, 110, 120])/100
vix_S0 = 19.81
vix_K = vix_S0 * vix_k

In [8]:
N = len(vix_T); drift = np.zeros(N);
R = np.zeros(N); Q = np.zeros(N);

for i in range(N):
    T = vix_T[i]; F = vix_f[i];
    drift[i] = -np.log(vix_S0/F)/T
    R[i] = rf.r(T)
    Q[i] = R[i] - drift[i]

## SPX data and functions

In [9]:
def dW(n_steps,N_sims):
    w = np.random.normal(0, 1, (n_steps, N_sims))
    #Antithetic variates
    w = np.concatenate((w, -w), axis = 1)
    return w

In [10]:
def price(S, K, r, q, T, opt):
    # opt stands for the option type: True = call, False = put
    N = len(K)
    P = np.zeros(N)
    for i in range(N):
        if opt[i]:
            P[i] = np.mean(np.maximum(S-K[i],0)*np.exp(-(r-q)*T))
        else:
            P[i] = np.mean(np.maximum(K[i]-S,0)*np.exp(-(r-q)*T))
    return P

In [11]:
def global_reduction(rho,H,eps,T,a_k_part,S0,strike_array,n_steps,N_sims,w1,steps,maturities):
    
    spine_k_order = 3
    
    eta_tild = eps**(H-0.5)
    kappa_tild = (0.5-H)/eps
    
    a_0,a_1,a_3,a_5 = a_k_part
    a_k = np.array([a_0,a_1,0,a_3,0,a_5])

    dt = T/n_steps
    tt = np.linspace(0., T, n_steps + 1)
    
    r = np.zeros(len(tt))
    r[:steps[0]] = rf.r(tt[steps[0]])
    r[steps[0]:] = rf.r(tt[steps[0]:])
    q = list()
    for i in range(len(tt)):
        if i <= steps[0]:
            q.append(iq.q(tt[steps[0]]))
        else:
            q.append(iq.q(tt[i]))
    q = np.array(q)

    exp1 = np.exp(kappa_tild*tt)
    exp2 = np.exp(2*kappa_tild*tt)

    diff_exp2 = np.concatenate((np.array([0.]),np.diff(exp2)))
    std_vec = np.sqrt(diff_exp2/(2*kappa_tild))[:,np.newaxis] #to be broadcasted columnwise 
    exp1 = exp1[:,np.newaxis] 
    X = (1/exp1)*(eta_tild*np.cumsum(std_vec*w1, axis = 0)) 
    Xt = np.array(X[:-1])
    del X
    
    tt = tt[:-1]
    std_X_t = np.sqrt(eta_tild**2/(2*kappa_tild)*(1-np.exp(-2*kappa_tild*tt)))
    n = len(a_k)
    
    cauchy_product = np.convolve(a_k,a_k)
    normal_var = np.sum(cauchy_product[np.arange(0,2*n,2)].reshape(-1,1)*std_X_t**(np.arange(0,2*n,2).reshape(-1,1))*\
        scipy.special.factorial2(np.arange(0,2*n,2).reshape(-1,1)-1),axis=0)
    
    f_func = horner_vector(a_k[::-1], len(a_k), Xt)
        
    del Xt
    
    fv_curve = vc.variance_curve(tt).reshape(-1,1)

    volatility = f_func/np.sqrt(normal_var.reshape(-1,1))
    del f_func
    volatility = np.sqrt(fv_curve)*volatility
    
    ST1 = list()
    logS1 = np.log(S0)
    for i in range(w1.shape[0]-1):
        logS1 = logS1 - 0.5*dt*(volatility[i]*rho)**2 + np.sqrt(dt)*rho*volatility[i]*w1[i+1] + rho**2*(r[i]-q[i])*dt
        if i in steps:
            ST1.append(np.exp(logS1))
    del w1
    ST1.append(np.exp(logS1))
    ST1 = np.array(ST1)
    del logS1 

    int_var = np.sum(volatility[:-1,]**2*dt,axis=0)
    Q = np.max(int_var)+1e-9
    del volatility
    
    P = list()
    
    for i in range(len(steps)):
        T_aux = maturities[i]
        r = rf.r(T_aux); q = iq.q(maturities[i])
        
        X = (bs.BSCall(ST1[i], strike_array.reshape(-1,1), T_aux, r, q, np.sqrt((1-rho**2)*int_var/T))).T
        Y = (bs.BSCall(ST1[i], strike_array.reshape(-1,1), T_aux, r, q, np.sqrt(rho**2*(Q-int_var)/T))).T
        eY = (bs.BSCall(S0, strike_array.reshape(-1,1), T_aux, r, q, np.sqrt(rho**2*Q/T))).T

        c = []
        for i in range(strike_array.shape[0]):
            cova = np.cov(X[:,i]+10,Y[:,i]+10)[0,1]
            varg = np.cov(X[:,i]+10,Y[:,i]+10)[1,1]
            if (cova or varg)<1e-8:
                temp = 1e-40
            else:
                temp = np.nan_to_num(cova/varg,1e-40)
            temp = np.minimum(temp,2)
            c.append(temp)
        c = np.array(c)

        call_mc_cv1 = X-c*(Y-eY)
        P.append(np.average(call_mc_cv1,axis=0))
    #std_mc_cv1 = np.std(call_mc_cv1,axis=0)
    
    return np.array(P)

In [12]:
spx_S0 = 4017.8
t0 = "23 Jan 2023"

IV_df = pd.read_csv("hist_spx.csv")
moneyness = np.array([80.0,90.0,95.0,97.5,100.0,102.5,105.0,110.0,120.0])
maturities = np.array(IV_df['Exp Date']).flatten()
spx_IV = np.array(IV_df.drop(columns = 'Exp Date'))/100.

spx_K = moneyness*spx_S0/100

In [29]:
start_time = time.time()

nr = len(maturities); nc = len(spx_K);
inp = np.array([-0.65, 0.2, 0.02, 1, 0.01, 0.02, 0.05]) # Parameter array [rho, H, eps, a0, a1, a3, a5]
bnds = ([-0.999, -0.1, 1e-10, 0, 0, 0, 0],[-1e-9, 0.5, 0.5, np.inf, np.inf, np.inf, np.inf])
N = 12500; n = 3650;
T = maturities[-1];
steps = np.ceil(n*maturities/T).astype(int)
option_type = np.ones(nc)
NR = len(vix_T); NC = len(vix_K); n_vix = 2000;
c1 = 1; c2 = 1; c3 = 1;

w = np.concatenate((np.zeros([1,N*2]), dW(n, N)))
    
def h(x):
    
    t = time.time()
    
    np.random.seed(0)
    iv = np.zeros([nr,nc])
        
    rho, H, eps, a0, a1, a3, a5 = x
    a_k = np.array([a0, a1, a3, a5])

    P = global_reduction(rho, H, eps, T, a_k, spx_S0, spx_K, n, N, w, steps, maturities)
    
    for i in range(nr):
        T_aux = maturities[i]
        r = rf.r(T_aux); q = iq.q(T_aux);
        iv[i,:] = bs.BSImpliedVol(spx_S0, spx_K, T_aux, r, q, P[i], Option_type = 1, toll = 1e-5)
    
    print(f'Function f execution time: {time.time()-t: .2f}s')

    return iv

def g(x):
    
    t = time.time()
    
    rho,H,eps,a0,a1,a3,a5 = x
    a_k = np.array([a0, a1, a3, a5])
    iv = np.zeros([NR, NC])
    
    for i in range(NR):
        T = vix_T[i]; r = R[i]; q = Q[i];
        P = vix_iv(H, eps, T, a_k, vix_K, r, q, n_vix)
        iv[i,:] = bs.BSImpliedVol(vix_S0, vix_K, T, r, q, P, 1, 1e-10)
        
    print(f'Function g execution time: {time.time()-t: .2f}s')
    
    return iv

def z(x):
    rho,H,eps,a0,a1,a3,a5 = x
    a_k = np.array([a0, a1, a3, a5])
    ft = np.zeros(NR)
        
    for i in range(NR):
        T = vix_T[i]; r = R[i]; q = Q[i];
        ft[i] = vix_futures(H, eps, T, a_k, vix_k, r, q, n_vix)
        
    return ft

def f(x):
    residuals = np.append(c1*(h(x).flatten() - spx_IV.flatten()), c2*(g(x).flatten() - vix_IV.flatten()))
    residuals = np.append(residuals, c3*(z(x)-vix_f))
    return residuals

result = ls(f, inp, bounds = bnds, max_nfev = 10, ftol = 1e-10, gtol = 1e-10, xtol = 1e-10)
model_param = result.x
    
model_iv_spx = h(result.x)
model_iv_vix = g(result.x)
model_futures = z(result.x)

print(f'\nTotal execution time: {(time.time() - start_time)/60: .2f} minutes\n')

print(f'IV SPX mean percentage error: {np.mean(np.abs((model_iv_spx-spx_IV)/spx_IV))*100: .4f}%')
print(f'IV VIX mean percentage error: {np.mean(np.abs((model_iv_vix-vix_IV)/vix_IV))*100: .4f}%')
print(f'VIX futures mean percentage error: {np.mean(np.abs((model_futures-vix_f)/vix_f))*100: .4f}%')

Function f execution time:  43.08s
Function g execution time:  4.20s
Function f execution time:  38.63s
Function g execution time:  4.25s
Function f execution time:  38.72s
Function g execution time:  4.27s
Function f execution time:  40.12s
Function g execution time:  4.25s
Function f execution time:  37.30s
Function g execution time:  4.11s
Function f execution time:  35.10s
Function g execution time:  4.14s
Function f execution time:  34.05s
Function g execution time:  4.12s
Function f execution time:  36.58s
Function g execution time:  4.14s
Function f execution time:  37.18s
Function g execution time:  7.03s
Function f execution time:  32.17s
Function g execution time:  6.81s
Function f execution time:  41.38s
Function g execution time:  5.83s
Function f execution time:  43.56s
Function g execution time:  6.09s
Function f execution time:  40.52s
Function g execution time:  5.82s
Function f execution time:  38.08s
Function g execution time:  5.89s
Function f execution time:  36.82s

Function f execution time:  46.27s
Function g execution time:  8.88s
Function f execution time:  32.75s
Function g execution time:  7.67s
Function f execution time:  34.30s
Function g execution time:  7.16s
Function f execution time:  39.73s
Function g execution time:  7.27s
Function f execution time:  35.82s
Function g execution time:  8.13s
Function f execution time:  37.42s
Function g execution time:  6.97s
Function f execution time:  31.65s
Function g execution time:  7.04s
Function f execution time:  31.68s
Function g execution time:  7.03s
Function f execution time:  36.65s
Function g execution time:  6.97s
Function f execution time:  28.92s
Function g execution time:  6.95s
Function f execution time:  33.13s
Function g execution time:  6.94s
Function f execution time:  31.91s
Function g execution time:  7.03s
Function f execution time:  34.34s
Function g execution time:  6.97s
Function f execution time:  31.16s
Function g execution time:  7.00s
Function f execution time:  32.91s

In [28]:
print(f'IV SPX mean percentage error: {np.mean(np.abs((model_iv_spx-spx_IV)/spx_IV))*100: .4f}%')
print(f'IV VIX mean percentage error: {np.mean(np.abs((model_iv_vix-vix_IV)/vix_IV))*100: .4f}%')
print(f'VIX futures mean percentage error: {np.mean(np.abs((model_futures-vix_f)/vix_f))*100: .4f}%')

IV SPX mean percentage error:  7.7591%
IV VIX mean percentage error:  18.3786%
VIX futures mean percentage error:  0.4339%


In [25]:
df = pd.DataFrame(model_param.reshape((1,7)), columns = ["rho", "H", "eps", "a0", "a1", "a3", "a5"])
df.to_csv("Joint_param.csv", index = False)